![Photo by Stephen Phillips - Hostreviews.co.uk on UnSplash](https://cf.bstatic.com/xdata/images/hotel/max1024x768/408003083.jpg?k=c49b5c4a2346b3ab002b9d1b22dbfb596cee523b53abef2550d0c92d0faf2d8b&o=&hp=1){fig-align="center" width=50%}


# Import data

In [1]:
import time
from pathlib import Path

import numpy as np
import pandas as pd
from data import utils
from lets_plot import *
from lets_plot.mapping import as_discrete

LetsPlot.setup_html()

**Goal**:
- Identify what basic pre-processing steps need to be taken before uploading the data to a database

# Select Columns to Retain Based on the Quantity of Missing Values


In the realm of web scraping, managing the sheer volume of data is often the initial hurdle to conquer. It's not so much about deciding what data to collect but rather what data to retain. As we delve into the vast realm of the Imoweb website, we are met with a plethora of listings, each offering a unique set of information.

For many of these listings, there are commonalities – details like location and price tend to be constants. However, interspersed among them are those one-of-a-kind nuggets of information, such as the number of swimming pools available. While these specific details can certainly be vital in assessing the value of certain listings, the downside is that they can lead to a sparse dataset.

Currently, our primary objective is to pinpoint which features are prevalent across the board, drawing insights from a pre-scraped dataset comprising around 1000 ads. Once we've identified these common denominators, we can streamline our data collection process by retaining these key attributes while discarding the less likely occurrences.

In [2]:
for filename in utils.Configuration.RAW_DATA_PATH.glob("*.gzip"):
    if "data" in filename.stem:
        df = pd.read_parquet(filename)
print(df.shape)
df.head()

(60, 50)


,Available as of,Construction year,Building condition,Street frontage width,Number of frontages,Covered parking spaces,Outdoor parking spaces,Surroundings type,Living area,Living room surface,...,Cadastral income,Tenement building,Address,External reference,day_of_retrieval,ad_url,Website,As built plan,Office,Dining room
0,After signing the deed,1971,Just renovated,12 m,4,2,5,"Living area (residential, urban or rural)",197 m² square meters,47 m² square meters,...,"€ 1,279 1279 €",No,Stationstraat 30 9600 - Ronse,5411439,2023-09-23 17:11:29.870179,https://www.immoweb.be/en/classified/villa/for...,None,None,None,None
1,After signing the deed,1949,To renovate,18 m,3,1,None,Isolated,139 m² square meters,10 m² square meters,...,€ 689 689 €,No,Rue de la Wallonie 2A 4680 - Oupeye,5534704,2023-09-23 17:11:29.995637,https://www.immoweb.be/en/classified/house/for...,http://www.nigel-immo.be,None,Yes,Yes
2,After signing the deed,1920,Good,5.5 m,2,None,None,Urban,200 m² square meters,26 m² square meters,...,€ 917 917 €,No,Sint-Denijslaan 1 9000 - Gent,5531386,2023-09-23 17:11:32.203024,https://www.immoweb.be/en/classified/mansion/f...,http://www.immodavinci.be,No,Yes,None
3,After signing the deed,1937,To renovate,8 m,2,None,None,Urban,230 m² square meters,None,...,"€ 1,623 1623 €",No,Hoogstraat 20 9340 - Lede,5535368,2023-09-23 17:11:34.716784,https://www.immoweb.be/en/classified/house/for...,http://www.immoderas.be,None,Yes,None
4,Depending on the tenant,1900,Good,6.5 m,2,None,None,Urban,360 m² square meters,34 m² square meters,...,"€ 1,621 1621 €",No,"Mechelsesteenweg,157 2018 - Antwerpen",5534431,2023-09-23 17:11:34.857664,https://www.immoweb.be/en/classified/mansion/f...,None,None,None,None


In [3]:
def pre_process_dataframe(df):
    return (
        df.rename(columns=lambda column: column.lower().replace(" ", "_"))
        .rename(columns=lambda column: column.lower().replace("&", ""))
        .rename(columns=lambda column: column.lower().replace(",", ""))
        .rename(columns={"co₂_emission": "co2_emission"})
        .assign(
            construction_year=lambda df: df.construction_year.str.extract(
                r"(\d+)"
            ).astype("float16"),
            street_frontage_width=lambda df: df.street_frontage_width.str.extract(
                r"(\d+)"
            ).astype("float16"),
            number_of_frontages=lambda df: df.number_of_frontages.str.extract(
                r"(\d+)"
            ).astype("float16"),
            covered_parking_spaces=lambda df: df.covered_parking_spaces.str.extract(
                r"(\d+)"
            ).astype("float16"),
            outdoor_parking_spaces=lambda df: df.outdoor_parking_spaces.str.extract(
                r"(\d+)"
            ).astype("float16"),
            living_area=lambda df: df.living_area.str.extract(r"(\d+)").astype(
                "float16"
            ),
            living_room_surface=lambda df: df.living_room_surface.str.extract(
                r"(\d+)"
            ).astype("float16"),
            kitchen_surface=lambda df: df.kitchen_surface.str.extract(r"(\d+)").astype(
                "float16"
            ),
            bedrooms=lambda df: df.bedrooms.str.extract(r"(\d+)").astype("float16"),
            bedroom_1_surface=lambda df: df.bedroom_1_surface.str.extract(
                r"(\d+)"
            ).astype("float16"),
            bedroom_2_surface=lambda df: df.bedroom_2_surface.str.extract(
                r"(\d+)"
            ).astype("float16"),
            bedroom_3_surface=lambda df: df.bedroom_3_surface.str.extract(
                r"(\d+)"
            ).astype("float16"),
            bathrooms=lambda df: df.bathrooms.str.extract(r"(\d+)").astype("float16"),
            toilets=lambda df: df.toilets.str.extract(r"(\d+)").astype("float16"),
            surface_of_the_plot=lambda df: df.surface_of_the_plot.str.extract(
                r"(\d+)"
            ).astype("float16"),
            width_of_the_lot_on_the_street=lambda df: df.width_of_the_lot_on_the_street.str.extract(
                r"(\d+)"
            ).astype(
                "float16"
            ),
            garden_surface=lambda df: df.garden_surface.str.extract(r"(\d+)").astype(
                "float16"
            ),
            primary_energy_consumption=lambda df: df.primary_energy_consumption.str.extract(
                r"(\d+)"
            ).astype(
                "float16"
            ),
            co2_emission=lambda df: df.co2_emission.str.extract(r"(\d+)").astype(
                "float16"
            ),
            yearly_theoretical_total_energy_consumption=lambda df: df.yearly_theoretical_total_energy_consumption.str.extract(
                r"(\d+)"
            ).astype(
                "float32"
            ),
            price=lambda df: df.price.str.replace(",", "")
            .str.extract(r"(\d+)")
            .astype("float32"),
            cadastral_income=lambda df: df.cadastral_income.str.replace(",", "")
            .str.extract(r"(\d+)")
            .astype("float32"),
            basement=lambda df: df.basement.map(
                {"Yes": True, None: False, "No": False}
            ),
            furnished=lambda df: df.furnished.map(
                {"Yes": True, None: False, "No": False}
            ),
            gas_water__electricity=lambda df: df.gas_water__electricity.map(
                {"Yes": True, None: False, "No": False}
            ),
            double_glazing=lambda df: df.double_glazing.map(
                {"Yes": True, None: False, "No": False}
            ),
            planning_permission_obtained=lambda df: df.planning_permission_obtained.map(
                {"Yes": True, None: False, "No": False}
            ),
            subdivision_permit=lambda df: df.subdivision_permit.map(
                {"Yes": True, None: False, "No": False}
            ),
            possible_priority_purchase_right=lambda df: df.possible_priority_purchase_right.map(
                {"Yes": True, None: False, "No": False}
            ),
            proceedings_for_breach_of_planning_regulations=lambda df: df.proceedings_for_breach_of_planning_regulations.map(
                {"Yes": True, None: False, "No": False}
            ),
            tenement_building=lambda df: df.tenement_building.map(
                {"Yes": True, None: False, "No": False}
            ),
            as_built_plan=lambda df: df.as_built_plan.map(
                {
                    "Yes, conform": True,
                    None: False,
                    "No": False,
                }
            ),
            office=lambda df: df.office.map({"Yes": True, None: False, "No": False}),
            dining_room=lambda df: df.dining_room.map(
                {"Yes": True, None: False, "No": False}
            ),
            flood_zone_type=lambda df: df.flood_zone_type.map(
                {
                    "Non flood zone": False,
                    None: False,
                    "No": False,
                    "Possible flood zone": True,
                }
            ),
            connection_to_sewer_network=lambda df: df.connection_to_sewer_network.map(
                {
                    "Connected": True,
                    None: False,
                    "Not connected": False,
                }
            ),
            tv_cable=lambda df: df.tv_cable.map(
                {
                    "Yes": True,
                    None: False,
                }
            ),
        )
    )


pre_processed_dataframe = pre_process_dataframe(df)
pre_processed_dataframe

,available_as_of,construction_year,building_condition,street_frontage_width,number_of_frontages,covered_parking_spaces,outdoor_parking_spaces,surroundings_type,living_area,living_room_surface,...,cadastral_income,tenement_building,address,external_reference,day_of_retrieval,ad_url,website,as_built_plan,office,dining_room
0,After signing the deed,1971.0,Just renovated,12.0,4.0,2.0,5.0,"Living area (residential, urban or rural)",197.0,47.0,...,1279.0,False,Stationstraat 30 9600 - Ronse,5411439,2023-09-23 17:11:29.870179,https://www.immoweb.be/en/classified/villa/for...,None,False,False,False
1,After signing the deed,1949.0,To renovate,18.0,3.0,1.0,NaN,Isolated,139.0,10.0,...,689.0,False,Rue de la Wallonie 2A 4680 - Oupeye,5534704,2023-09-23 17:11:29.995637,https://www.immoweb.be/en/classified/house/for...,http://www.nigel-immo.be,False,True,True
2,After signing the deed,1920.0,Good,5.0,2.0,NaN,NaN,Urban,200.0,26.0,...,917.0,False,Sint-Denijslaan 1 9000 - Gent,5531386,2023-09-23 17:11:32.203024,https://www.immoweb.be/en/classified/mansion/f...,http://www.immodavinci.be,False,True,False
3,After signing the deed,1937.0,To renovate,8.0,2.0,NaN,NaN,Urban,230.0,NaN,...,1623.0,False,Hoogstraat 20 9340 - Lede,5535368,2023-09-23 17:11:34.716784,https://www.immoweb.be/en/classified/house/for...,http://www.immoderas.be,False,True,False
4,Depending on the tenant,1900.0,Good,6.0,2.0,NaN,NaN,Urban,360.0,34.0,...,1621.0,False,"Mechelsesteenweg,157 2018 - Antwerpen",5534431,2023-09-23 17:11:34.857664,https://www.immoweb.be/en/classified/mansion/f...,None,False,False,False
5,None,1925.0,As new,5.0,2.0,NaN,NaN,None,178.0,30.0,...,518.0,False,Plantin en Moretuslei 135A 2140 - Borgerhout,11351 - 3966,2023-09-23 17:11:35.080346,https://www.immoweb.be/en/classified/mansion/f...,http://www.immovasta.be,False,False,False
6,None,1918.0,To be done up,12.0,2.0,3.0,2.0,Urban,148.0,NaN,...,936.0,False,Chaussee de Charleroi 59 B 5030 - Gembloux,441 - 93363974,2023-09-23 17:11:35.208188,https://www.immoweb.be/en/classified/house/for...,http://www.trevitessier.be,False,False,False
7,After signing the deed,2017.0,As new,6.0,3.0,NaN,2.0,Urban,145.0,27.0,...,833.0,False,Hoogstraat 20 9340 - Lede,5531684,2023-09-23 17:11:35.374734,https://www.immoweb.be/en/classified/house/for...,http://www.immoderas.be,False,False,False
8,After signing the deed,1929.0,As new,8.0,2.0,NaN,1.0,"Living area (residential, urban or rural)",300.0,39.0,...,2791.0,False,Avenue de Tervuren 113 1040 - Etterbeek,5532305,2023-09-23 17:11:35.544038,https://www.immoweb.be/en/classified/house/for...,None,False,False,False
9,After signing the deed,1967.0,Good,6.0,2.0,NaN,NaN,Urban,249.0,30.0,...,1026.0,True,Hoogstraat 20 9340 - Lede,5523589,2023-09-23 17:11:35.756325,https://www.immoweb.be/en/classified/mixed-use...,http://www.immoderas.be,False,True,False


In [4]:
(
    df["Construction year"]
    .str.extract(r"([^a-zA-Z])", expand=True)  # NON-matching alphabetical characters
    .value_counts()
)

1    42
2     6
Name: count, dtype: int64

# Assessing Feature Cardinality

In [5]:
# Assuming df is your DataFrame
number_unique_entries = {
    "column_name": pre_processed_dataframe.columns.tolist(),
    "column_dtype": [
        pre_processed_dataframe[col].dtype for col in pre_processed_dataframe.columns
    ],
    "unique_values_pct": [
        pre_processed_dataframe[col].nunique()
        for col in pre_processed_dataframe.columns
    ],
}

result_df = (
    pd.DataFrame(number_unique_entries)
    .sort_values("unique_values_pct")
    .assign(
        unique_values_pct=lambda x: x.unique_values_pct.div(df.shape[0])
        .mul(100)
        .round(1)
    )
    .pipe(
        lambda df: ggplot(df, aes("unique_values_pct", "column_name"))
        + geom_bar(stat="identity", orientation="y")
        + labs(
            title="Assessing Feature Cardinality",
            subtitle=""" Features with a Low Cardinality (Less than 10 Distinct Values) Can Be  Utilized as Categorical Variables, 
            while Those with Higher Cardinality, typically represented as floats or integers, May Be Employed as They Are
            """,
            x="Percentage of Unique Values per Feature",
            y="",
            caption="https://www.immoweb.be/",
        )
        + theme(
            plot_subtitle=element_text(
                size=12, face="italic"
            ),  # Customize subtitle appearance
            plot_title=element_text(size=15, face="bold"),  # Customize title appearance
        )
        + ggsize(800, 1000)
    )
)
result_df